# Ex1) P2P Task

In [75]:
from time import sleep
from tasho.templates.P2P import P2P
from tasho import TaskModel, default_mpc_options
from tasho.templates.Regularization import Regularization
from tasho.Variable import Variable
from robotshyu import Robot as rob
from tasho.OCPGenerator import OCPGenerator
import numpy as np
import casadi as cs
import tasho
from tasho.ConstraintExpression import ConstraintExpression
from tasho.Expression import Expression
from tasho.MPC import MPC 
from tasho import environment as env

In [76]:
robot = rob.Robot("mmo_500_ppr")
link_name = 9
goal_pose = Variable(robot.name, "goal_pose", "magic_number", (4,4), np.array(
        [[0, 1, 0, 0.6], [1, 0, 0, 0.0], [0, 0, -1, 0.25], [0, 0, 0, 1]]
    ))
q0 = [1.0193752249977548, -0.05311582280659044, -2.815452580946695,
    1.3191046402052224, 0.8582660722530533, 1.3988994390898029,1.8226311094569714, 0.0, 0.0]
robot.set_joint_acceleration_limits(lb=-360*3.14159/180, ub=360*3.14159/180)
ndof = robot.ndof
q_current = Variable(robot.name, "jointpos_init", 'magic_number', (ndof, 1), q0)

Loading robot params from json ...
Loaded 9-DoF robot: mmo_500_ppr


In [77]:
task_P2P = P2P(robot, link_name, goal_pose, q_current, 0.001)

In [78]:
task_P2P.write_task_graph("P2P_Task.svg")

In [79]:
reg_jacc = task_P2P.constraint_expressions['reg_qdd_'+robot.name].change_weight(0.1)
task_P2P.add_path_constraints(Regularization(task_P2P.variables['qd_'+robot.name], 1))

In [80]:
task_P2P.write_task_graph("P2P_Task2.svg")

In [81]:
horizon_steps = 10
horizon_period = 3.0
OCP_gen = OCPGenerator(task_P2P, False, {"time_period": horizon_period, "horizon_steps":horizon_steps})
q_ocp = OCP_gen.stage_tasks[0].variables['q_'+robot.name].x
OCP_gen.tc.set_initial(q_ocp, q0)
OCP_gen.tc.set_ocp_solver(
    "ipopt", 
    # {"ipopt":{"linear_solver":"ma27"}}
)
OCP_gen.tc.solve_ocp()

This is Ipopt version 3.12.7, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:      675
Number of nonzeros in inequality constraint Jacobian.:      277
Number of nonzeros in Lagrangian Hessian.............:      208

Total number of variables............................:      288
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      210
Total number of inequality constraints...............:      271
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:      270
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 1.62e+00 1.25e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00   0
   1

In [82]:
t_grid, qsol = OCP_gen.tc.sol_sample(q_ocp)
_, q_dot_sol = OCP_gen.tc.sol_sample(OCP_gen.stage_tasks[0].variables['qd_'+robot.name].x)

from tasho import WorldSimulator
import pybullet as p

obj = WorldSimulator.WorldSimulator()

    # Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
robotID = obj.add_robot(position, orientation, robot.name)
joint_indices = [0, 1, 2, 13, 14, 15, 16, 17, 18]

obj.resetJointState(robotID, joint_indices, q0)

for i in range(horizon_steps + 1):
    sleep(horizon_period*0.5/horizon_steps)
    obj.resetJointState(
        robotID, joint_indices, qsol[i]
    )

sleep(0.5)
obj.end_simulation()

Ending simulation


# Ex2) Object picking MPC

In [59]:
# Define initial conditions of the robot
q0_val = [-1.0, -0.523598, 3.14, 3.14, 0, -0.523598, 0.0, 0.0, 0.0]
qd0_val = [0] * robot.ndof

################################################
# Task spacification - Approximation to object
################################################

# Select prediction horizon and sample time for the MPC execution
horizon_size = 10
t_mpc = 0.05

q_init = Variable(robot.name, 'q_init', 'parameter', (robot.ndof,1))
qd_init = Variable(robot.name, 'qd_init', 'parameter', (robot.ndof,1))
goal_pose = Variable(robot.name, "goal_pose", 'parameter', (4,4))

task_P2P = P2P(robot, link_name, goal_pose, q_init, 0.001)
task_P2P.write_task_graph("P2P_Task4.svg")

In [60]:
task_P2P.remove_initial_constraints(task_P2P.constraint_expressions['stationary_qd_mmo_500_ppr'])
task_P2P.remove_terminal_constraints('rot_con_pose_9_mmo_500_ppr_vs_goal',
                                    'trans_con_pose_9_mmo_500_ppr_vs_goal')
task_P2P.write_task_graph("P2P_Task5.svg")

In [61]:
task_P2P.add_initial_constraints(ConstraintExpression(robot.name, "eq", Expression(robot.name, "err_qd_qinit", lambda a, b : a - b, qd_init, task_P2P.variables['qd_mmo_500_ppr']),
                             "hard", reference = 0))
task_P2P.add_path_constraints(Regularization(task_P2P.expressions['trans_error_pose_9_mmo_500_ppr_vs_goal'], 1.0, norm = "L1"),
                            Regularization(task_P2P.expressions['ax_ang_error_pose_9_mmo_500_ppr_vs_goal'], 1), 
                            Regularization(task_P2P.variables['qd_mmo_500_ppr'], 1e-4))

task_P2P.write_task_graph("P2P_Task6.svg")

In [62]:
pOCP = OCPGenerator(task_P2P, False, {"time_period":horizon_size*t_mpc, "horizon_steps":horizon_size})

tc = pOCP.tc

################################################
# Set solver and discretization options
################################################
tc.set_ocp_solver("ipopt", {"ipopt": {"print_level": 5,"tol": 1e-3}})
# tc.set_ocp_solver("ipopt", {"ipopt": {"print_level": 0,"tol": 1e-3, "linear_solver":"ma27"}}) #use this if you have hsl


################################################
# Set parameter values
################################################
q = pOCP.stage_tasks[0].variables['q_mmo_500_ppr'].x
qd = pOCP.stage_tasks[0].variables['qd_mmo_500_ppr'].x
q_0 = pOCP.stage_tasks[0].variables['q_init_mmo_500_ppr'].x
qd_0 = pOCP.stage_tasks[0].variables['qd_init_mmo_500_ppr'].x
goal_pose = pOCP.stage_tasks[0].variables['goal_pose_mmo_500_ppr'].x
goal_pose_val =  cs.vertcat(
    cs.hcat([0, 1, 0, 0.5]),
    cs.hcat([1, 0, 0, 0.0]),
    cs.hcat([0, 0, -1, 0.25]),
    cs.hcat([0, 0, 0, 1]),
)
tc.set_initial(q, q0_val)
tc.set_value(goal_pose, goal_pose_val)
tc.set_value(q_0, q0_val)
tc.set_value(qd_0, qd0_val)

# Add an output port for joint velocities as well
tc.tc_dict["out_ports"].append({"name":"port_out_qd_mmo_500_ppr", "var":"qd_mmo_500_ppr", "desc": "output port for the joint velocities"})

# Add a monitor for termination criteria
tc.add_monitor({"name":"termination_criteria", "expression":cs.sqrt(cs.sumsqr(pOCP.stage_tasks[0].expressions["trans_error_pose_9_mmo_500_ppr_vs_goal"].x)) - 2e-2, "reference":0, "lower":True, "initial":True})

mpc_options = default_mpc_options.get_default_mpc_options()


tc.ocp_solver = "ipopt"
tc.ocp_options = mpc_options["ipopt_hsl"]["options"]
tc.mpc_solver = tc.ocp_solver
tc.mpc_options = tc.ocp_options
tc.set_ocp_solver("ipopt", tc.mpc_options)
import os; os.system("export OMP_NUM_THREADS = 1")

sol = tc.solve_ocp()

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 219.00us ( 18.25us) 221.41us ( 18.45us)        12
       nlp_g  | 130.00us ( 10.83us) 127.25us ( 10.60us)        12
    nlp_grad  |  78.00us ( 78.00us)  77.45us ( 77.45us)         1
  nlp_grad_f  | 535.00us ( 44.58us) 534.14us ( 44.51us)        12
  nlp_hess_l  |   2.94ms (267.45us)   2.94ms (267.58us)        11
   nlp_jac_g  | 183.00us ( 15.25us) 182.74us ( 15.23us)        12
       total  |   8.25ms (  8.25ms)   8.23ms (  8.23ms)         1


sh: 1: export: : bad variable name


In [63]:
vars_db = tc.generate_MPC_component("./", {"ocp_cg_opts":{"save":True, "codegen":False, "jit":False}, "mpc":True, "mpc_cg_opts":{"save":True, "codegen":False, "jit":False}})

@1=(opti6_p_2[12:15]-fk_T(opti6_x_1){9}[12:15]), (sqrt(dot(@1, @1))-0.02)
@1=(opti6_p_2[12:15]-fk_T(opti6_x_1){9}[12:15]), (sqrt(dot(@1, @1))-0.02)


In [64]:
MPC_component = MPC("mmo_500_ppr_obj_pickup", "./" + tc.name + ".json")

In [65]:
obj = WorldSimulator.WorldSimulator(bullet_gui=True)

# Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
mmoID = obj.add_robot(position, orientation, "mmo_500_ppr")

# Set environment
environment = env.Environment()
package_path = tasho.__path__[0]
cube1 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small.urdf")
environment.add_object(cube1, "cube")
table1 = env.Box(height = 0.3, position = [0.5, 0, 0], orientation = [0.0, 0.0, 0.7071080798594737, 0.7071054825112364], urdf =package_path+ "/models/objects/table.urdf")
environment.add_object(table1, "table1")

cube2 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small_col.urdf", fixed = True)
environment.add_object(cube2, "cube2")
environment.set_in_world_simulator(obj)
cubeID = environment.get_object_ID("cube")
cube2ID = environment.get_object_ID("cube2")

#p.resetBaseVelocity(cubeID, linearVelocity=[0, 0.8, 0])
# Determine number of samples that the simulation should be executed
no_samples = int(t_mpc / obj.physics_ts)
if no_samples != t_mpc / obj.physics_ts:
    print("[ERROR] MPC sampling time not integer multiple of physics sampling time")

# Correspondence between joint numbers in bullet and OCP
joint_indices = [0, 1, 2, 13, 14, 15, 16, 17, 18]

 # Begin the visualization by applying the initial control signal
ts, q_sol = tc.sol_sample(q, grid="control")
ts, q_dot_sol = tc.sol_sample(qd, grid="control")
obj.resetJointState(mmoID, joint_indices, q0_val)
obj.setController(
    mmoID, "velocity", joint_indices, targetVelocities=q_dot_sol[0]
)

In [73]:
# Execute the MPC loop
q_log = []
q_dot_log = []
predicted_pos_log = []

p.resetBaseVelocity(cubeID, linearVelocity=[0, 1, 0])

for i in range(horizon_size * 100):
    print("----------- MPC execution -----------")

    q_now = obj.readJointPositions(mmoID, joint_indices)
    qd_now = obj.readJointVelocities(mmoID, joint_indices)
        
    MPC_component.input_ports["port_inp_q_init_mmo_500_ppr"]["val"] = q_now
    MPC_component.input_ports["port_inp_qd_init_mmo_500_ppr"]["val"] = qd_now

    # Predict the position of the target object (cube)
    lin_vel, ang_vel = p.getBaseVelocity(cubeID)
    lin_vel = cs.DM(lin_vel)
    lin_pos, _ = p.getBasePositionAndOrientation(cubeID)
    lin_pos = cs.DM(lin_pos)
    time_to_stop = cs.norm_1(lin_vel) / 0.3
    predicted_pos = (
        cs.DM(lin_pos)
        + cs.DM(lin_vel) * time_to_stop
        - 0.5 * 0.5 * lin_vel / (cs.norm_1(lin_vel) + 1e-3) * time_to_stop ** 2
    )
    predicted_pos_log.append(predicted_pos.full())
    p.resetBasePositionAndOrientation(cube2ID, predicted_pos.full(), [0.0, 0.0, 0.0, 1.0])
    predicted_pos[2] += 0.06  # cube height
    print("Predicted position of cube", predicted_pos)
    predicted_pos_val = cs.vertcat(
        cs.hcat([0, 0, 1, predicted_pos[0]]),
        cs.hcat([0, 1, 0, predicted_pos[1]]),
        cs.hcat([-1, 0, 0, predicted_pos[2]]),
        cs.hcat([0, 0, 0, 1]),
    )

    MPC_component.input_ports["port_inp_goal_pose_mmo_500_ppr"]["val"] = cs.vec(predicted_pos_val)
        
    if i == 0:
        MPC_component.configMPC()

    MPC_component.runMPC()

    q_log.append(q_now)
    q_dot_log.append(qd_now)

    # Set control signal to the simulated robot
    qd_control_sig = MPC_component.output_ports["port_out_qd_mmo_500_ppr"]["val"].full()
    qdd_control_sig = (MPC_component.output_ports["port_out_qdd_mmo_500_ppr"]["val"] * t_mpc).full()
    obj.setController(
        mmoID, "velocity", joint_indices, targetVelocities=qd_control_sig + qdd_control_sig
    )

    # Simulate
    obj.run_simulation(no_samples)

    # Termination criteria
    if "termination_criteria_true" in MPC_component.event_output_port:
        break

----------- MPC execution -----------
Predicted position of cube [0.50000779228895, -0.44474842774257, 1.0877661494541]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 163.00us ( 27.17us) 161.33us ( 26.89us)         6
       nlp_g  |  87.00us ( 14.50us)  85.58us ( 14.26us)         6
    nlp_grad  |   1.73ms (  1.73ms)  98.55us ( 98.55us)         1
  nlp_grad_f  | 371.00us ( 61.83us) 371.71us ( 61.95us)         6
  nlp_hess_l  |   1.63ms (326.60us)   1.63ms (326.03us)         5
   nlp_jac_g  | 117.00us ( 19.50us) 118.30us ( 19.72us)         6
       total  |  24.43ms ( 24.43ms)   6.79ms (  6.79ms)         1
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 104.00us ( 26.00us) 102.99us ( 25.75us)         4
       nlp_g  |  48.00us ( 12.00us)  47.08us ( 11.77us)         4
    nlp_grad  | 100.00us (100.00us) 100.54us (100.54us)         1
  nlp_grad_f  | 248.00us ( 62.00us) 247.75us ( 61.94us)         4
  nlp_hess_l  | 976.00

----------- MPC execution -----------
Predicted position of cube [0.50000652081354, -0.20913268062005, 0.084990663717635]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  98.00us ( 24.50us)  98.94us ( 24.73us)         4
       nlp_g  |  48.00us ( 12.00us)  47.44us ( 11.86us)         4
    nlp_grad  | 108.00us (108.00us) 108.23us (108.23us)         1
  nlp_grad_f  | 236.00us ( 59.00us) 236.60us ( 59.15us)         4
  nlp_hess_l  |   1.01ms (337.00us)   1.01ms (337.63us)         3
   nlp_jac_g  |  69.00us ( 17.25us)  70.08us ( 17.52us)         4
       total  |  12.27ms ( 12.27ms)   4.28ms (  4.28ms)         1
----------- MPC execution -----------
Predicted position of cube [0.50000651333549, -0.2197966751763, 0.084989878070027]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  95.00us ( 23.75us)  95.48us ( 23.87us)         4
       nlp_g  |  53.00us ( 13.25us)  51.62us ( 12.90us)         4
    nlp_grad  |  93.00us ( 93.0

In [74]:
obj.end_simulation()

max(MPC_component.solver_times)

Ending simulation


0.01703619956970215